In [1]:
import pandas as pd
features = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/Working_DataFrame.csv')
target = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/CleanedFinal_Target_Data_Combined_resid_Trend')

In [2]:
# Ensure the features and target dataframes have similar lengths
min_length = min(len(features), len(target))
features = features.iloc[:min_length]
target = target.iloc[:min_length]

# Print the lengths to verify
print(f"Length of features dataframe: {len(features)}")
print(f"Length of target dataframe: {len(target)}")

Length of features dataframe: 24574
Length of target dataframe: 24574


In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# Ensure the features and target dataframes are correctly assigned
features = features.drop(columns=['Datum'])
target = target['PM10_Combined_Trend_Residual']

# Add squared features
poly = PolynomialFeatures(degree=2, include_bias=False)
features_poly = poly.fit_transform(features)

# Split the data into train, test, and validate sets
train_data, temp_data, train_target, temp_target = train_test_split(features_poly, target, test_size=0.3, random_state=42)
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=42)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Define a smaller parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001],
    'eta0': [0.01, 0.1],
    'learning_rate': ['constant', 'optimal'],
    'penalty': ['l2', 'l1']
}

# Initialize the SGDRegressor
sgd = SGDRegressor(max_iter=1000, tol=1e-3)

# Initialize GridSearchCV with fewer cross-validation folds
grid_search = GridSearchCV(sgd, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Plot the progress bar
progress_bar = tqdm(total=100)
for i in range(100):
    progress_bar.update(1)
progress_bar.close()

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model on the test set
predictions = grid_search.predict(test_data)
mse = mean_squared_error(test_target, predictions)
print("Mean Squared Error on Test Set:", mse)

# Evaluate the model on the validation set
validate_predictions = grid_search.predict(validate_data)
validate_mse = mean_squared_error(validate_target, validate_predictions)
print("Mean Squared Error on Validation Set:", validate_mse)

# Expanding CV Fold Sizes
initial_train_size = int(0.5 * len(train_data))
test_size = int(0.1 * len(train_data))
num_folds = (len(train_data) - initial_train_size) // test_size

for fold in range(num_folds):
    start_train_size = initial_train_size + fold * test_size
    end_train_size = start_train_size + test_size

    X_train_fold = train_data[:end_train_size]
    y_train_fold = train_target[:end_train_size]
    X_test_fold = train_data[end_train_size:end_train_size + test_size]
    y_test_fold = train_target[end_train_size:end_train_size + test_size]

    model = SGDRegressor(**best_params, max_iter=1000, tol=1e-3)
    model.fit(X_train_fold, y_train_fold)
    fold_predictions = model.predict(X_test_fold)
    fold_mse = mean_squared_error(y_test_fold, fold_predictions)
    print(f"Fold {fold + 1}/{num_folds}, MSE: {fold_mse}")


Fitting 3 folds for each of 16 candidates, totalling 48 fits


Python(15911) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15912) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15913) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15914) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15915) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15916) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15917) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15918) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15919) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(15920) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
